# Common Neural Architectures    Hands-on snippets for shallow models, RBF networks, RBMs, CNNs, hierarchical feature stacks, and pretrained feature extractors.

In [ ]:
    # !pip install torch torchvision scikit-learn matplotlib    import torch    import torch.nn as nn    import torch.optim as optim    import torchvision    import torchvision.transforms as T    import matplotlib.pyplot as plt    import numpy as np    from sklearn.datasets import load_digits    from sklearn.neural_network import BernoulliRBM    from sklearn.model_selection import train_test_split    from sklearn.preprocessing import MinMaxScaler    torch.manual_seed(123)    

## Shallow Models    Logistic regression on a small dataset illustrates a shallow discriminative model.

In [ ]:
    # Generate synthetic binary data    X = torch.randn(400, 2)    true_w = torch.tensor([[1.5, -0.7]]).t()    logits = X @ true_w + 0.2    y = (torch.sigmoid(logits) > 0.5).long().squeeze()    model = nn.Sequential(nn.Linear(2, 1), nn.Sigmoid())  # single linear layer + sigmoid    criterion = nn.BCELoss()    opt = optim.SGD(model.parameters(), lr=0.5)    for _ in range(100):        pred = model(X).squeeze()        loss = criterion(pred, y.float())        opt.zero_grad(); loss.backward(); opt.step()    with torch.no_grad():        acc = ((model(X).squeeze() > 0.5) == y).float().mean().item()    print(f"Shallow logistic regression accuracy: {acc:.3f}")    

## Radial Basis Function (RBF) Networks    Build an RBF hidden layer manually: compute Gaussian responses around learned centers, then a linear readout.

In [ ]:
    class RBFNet(nn.Module):        def __init__(self, in_features=2, hidden_centers=10, sigma=1.0):            super().__init__()            self.centers = nn.Parameter(torch.randn(hidden_centers, in_features))            self.sigma = sigma            self.linear = nn.Linear(hidden_centers, 1)        def rbf(self, x):            # Compute squared Euclidean distance to each center            diff = x[:, None, :] - self.centers[None, :, :]            dist_sq = torch.sum(diff ** 2, dim=2)            return torch.exp(-dist_sq / (2 * self.sigma ** 2))        def forward(self, x):            phi = self.rbf(x)            return torch.sigmoid(self.linear(phi)).squeeze()    rbf_model = RBFNet(hidden_centers=8, sigma=1.5)    opt = optim.Adam(rbf_model.parameters(), lr=0.05)    bce = nn.BCELoss()    for _ in range(200):        pred = rbf_model(X)        loss = bce(pred, y.float())        opt.zero_grad(); loss.backward(); opt.step()    with torch.no_grad():        acc = ((rbf_model(X) > 0.5) == y).float().mean().item()    print(f"RBF network accuracy: {acc:.3f}")    

## Restricted Boltzmann Machines (RBM)    Use `BernoulliRBM` from scikit-learn on the digits dataset to learn unsupervised binary features.

In [ ]:
    digits = load_digits()    data = MinMaxScaler().fit_transform(digits.data)  # scale to [0,1] as Bernoulli RBM expects probabilities    X_train, X_test = train_test_split(data, test_size=0.2, random_state=42)    rbm = BernoulliRBM(n_components=64, learning_rate=0.06, batch_size=32, n_iter=15, random_state=0, verbose=0)    rbm.fit(X_train)    # Reconstruct a sample digit to see what the RBM captures    sample = X_test[0:1]    reconstruction = rbm.gibbs(sample)    plt.figure(figsize=(4,2))    plt.subplot(1,2,1); plt.imshow(sample.reshape(8,8), cmap='gray'); plt.title('Original'); plt.axis('off')    plt.subplot(1,2,2); plt.imshow(reconstruction.reshape(8,8), cmap='gray'); plt.title('RBM Recon'); plt.axis('off')    plt.show()    

## Convolutional Neural Networks (CNN)    Small CNN trained on synthetic image-like data using `torchvision.datasets.FakeData` to avoid downloads.

In [ ]:
    transform = T.Compose([T.ToTensor()])    train_data = torchvision.datasets.FakeData(size=200, image_size=(1, 28, 28), num_classes=10, transform=transform)    loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True)    class SmallCNN(nn.Module):        def __init__(self):            super().__init__()            self.features = nn.Sequential(                nn.Conv2d(1, 8, kernel_size=3, padding=1),                nn.ReLU(),                nn.MaxPool2d(2),                nn.Conv2d(8, 16, kernel_size=3, padding=1),                nn.ReLU(),                nn.MaxPool2d(2)            )            self.classifier = nn.Sequential(                nn.Flatten(),                nn.Linear(16 * 7 * 7, 32),                nn.ReLU(),                nn.Linear(32, 10)            )        def forward(self, x):            return self.classifier(self.features(x))    cnn = SmallCNN()    opt_cnn = optim.Adam(cnn.parameters(), lr=0.001)    ce = nn.CrossEntropyLoss()    for batch, (images, labels) in enumerate(loader):        logits = cnn(images)        loss = ce(logits, labels)        opt_cnn.zero_grad(); loss.backward(); opt_cnn.step()        if batch % 20 == 0:            print(f"Batch {batch}, loss {loss.item():.3f}")        if batch == 40:            break  # keep the demo fast    

## Hierarchical Feature Engineering    Inspect feature maps across layers to see hierarchy emerge.

In [ ]:
    sample_img, _ = train_data[0]    with torch.no_grad():        feat1 = cnn.features[0:2](sample_img.unsqueeze(0))  # after first conv+ReLU        feat2 = cnn.features(sample_img.unsqueeze(0))        # after two conv blocks    print("Input shape:", sample_img.unsqueeze(0).shape)    print("After first block:", feat1.shape)    print("After second block:", feat2.shape)    

## Pretrained Models    Load a pretrained ResNet18 and use it as a fixed feature extractor. Commented line shows how to download ImageNet weights.

In [ ]:
    from torchvision.models import resnet18, ResNet18_Weights    # weights = ResNet18_Weights.IMAGENET1K_V1  # uncomment to download actual pretrained weights    weights = None  # keep offline by default    pretrained_model = resnet18(weights=weights)    pretrained_model.eval()    for param in pretrained_model.parameters():        param.requires_grad = False  # freeze when using as feature extractor    dummy = torch.randn(1, 3, 224, 224)    with torch.no_grad():        features = pretrained_model.forward(dummy)    print("Feature vector shape:", features.shape)    